In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from digital_processing import plot_signal
import scipy.io



In [38]:
def load_pre_v7_3_mat(file_path):
    """
    Loads a MATLAB .mat file in pre-7.3 format.

    Args:
        file_path (str): Path to the .mat file.

    Returns:
        dict: Dictionary containing the data from the .mat file.
    """
    try:
        mat_data = scipy.io.loadmat(file_path)
        usable_data = {key: value for key, value in mat_data.items() if not key.startswith('__')}
        return usable_data
    except Exception as e:
        print(f"Error reading the .mat file: {e}")
        return None

#file_path = "C:\\Users\\reidw\\OneDrive - Milwaukee School of Engineering\\Documents\\School\\Senior Design\\EES-Algorithmic-Interpretation-System\\Data\\deidentified_Scrambled_EMG.mat"  # Replace with your file path
file_path = "../Data/deidentified_Scrambled_EMG.mat"
data = load_pre_v7_3_mat(file_path)
if data:
    print("Successfully loaded the .mat file:")
    for key, value in data.items():
        print(f"{key}: {value}")

Successfully loaded the .mat file:
deidentified_Scrambled_EMG: [[array([[-0.02539086, -0.02322523, -0.02392269, ...,  0.25195617,
           0.26842467,  0.23588683]])
  array([[ 0.13799806,  0.05239472,  0.12712758, ..., -0.76560487,
          -0.74918633, -0.6074814 ]])
  array([[ 0.02407035,  0.08621845,  0.03737655, ..., -0.29797667,
          -0.23661116, -0.22643767]])
  array([[-7.24018331e-02, -4.61039811e-02, -9.41160746e-03, ...,
           1.23843131e-02, -8.06422198e-05, -2.32376993e-02]])
  array([[-0.05215233, -0.07079666, -0.06974498, ..., -0.04849259,
          -0.04026978, -0.02983369]])
  array([[ 0.13382475,  0.17189861,  0.20306367, ..., -0.01930332,
           0.05645564, -0.01604348]])
  array([[-0.01879935,  0.12284746,  0.03799016, ..., -0.07236703,
          -0.07341698, -0.09176982]])
  array([[-9.10158545e-03, -3.09625797e-03, -1.36284149e-02, ...,
          -8.97078014e-03, -3.41903222e-03,  6.80478806e-05]])
  array([[-0.01516218, -0.00736223, -0.01159028, 

In [39]:
"""
x = np.arange(24399)
emg1 = mult_df.CH1.values
emg2 = mult_df.CH2.values
emg3 = mult_df.CH3.values
emg4 = mult_df.CH4.values
emg5 = mult_df.CH5.values
emg6 = mult_df.CH6.values
emg7 = mult_df.CH7.values
emg8 = mult_df.CH8.values
emg9 = mult_df.CH9.values
emg10 = mult_df.CH10.values
emg11 = mult_df.CH11.values
emg12 = mult_df.CH12.values
emg13 = mult_df.CH13.values
emg14 = mult_df.CH14.values
"""





'\nx = np.arange(24399)\nemg1 = mult_df.CH1.values\nemg2 = mult_df.CH2.values\nemg3 = mult_df.CH3.values\nemg4 = mult_df.CH4.values\nemg5 = mult_df.CH5.values\nemg6 = mult_df.CH6.values\nemg7 = mult_df.CH7.values\nemg8 = mult_df.CH8.values\nemg9 = mult_df.CH9.values\nemg10 = mult_df.CH10.values\nemg11 = mult_df.CH11.values\nemg12 = mult_df.CH12.values\nemg13 = mult_df.CH13.values\nemg14 = mult_df.CH14.values\n'

In [40]:
mat_data = scipy.io.loadmat("C:\Users\reidw\OneDrive - Milwaukee School of Engineering\Documents\School\Senior Design\EES-Algorithmic-Interpretation-System\Data\deidentified_Scrambled_EMG.mat")

fs = 4000; #Samples per second
stim_freq = 2; #Hz
t_int = 2; #Seconds between ramp-up
stim_pulses = 5; #5 pulses during 1 ramp-up int
stim_inc = 0.5; #0.5 V ramp 

stim_max = 10; #Max ramp-up voltage
stim_min = 1;  #Pre-activation Value Volts

stim_int = np.arange(stim_min, stim_max, stim_inc).tolist()
print(stim_int)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3093012885.py, line 1)

In [ ]:
   

def stim_timings(emg_data,fs):
    timing_data = emg_data[0]
    # timings = Stim_Pulse_data
    
    # Find the indices of the peaks
    peak_indices, _ = find_peaks(timing_data)

    # Print the peak values and their corresponding indices
    peak_times = peak_indices
    peak_values = timing_data[peak_indices]

    # Convert peak_times to milliseconds
    peak_times = peak_times/fs*1000



    return peak_times



def segment_emg_signals(emg_data, onset_timings, num_muscles):
    """
    Segment EMG signals based on the onset timings of stimulation.
    
    Parameters:
        emg_data (list of np.array): A list where each entry is an array of EMG data for a muscle.
        onset_timings (list): List of onset timings for each stimulation.
        num_muscles (int): Number of muscles.
    
    Returns:
        segmented_data (np.array): 3D array (m x n x l) where:
            - m is the number of stimulations,
            - n is the maximum time interval between consecutive stimulations,
            - l is the number of muscles.
    """
    
    #file_path = "deidentified_Scrambled_EMG.mat"
    
        # Calculate time intervals between consecutive stimulations
    time_intervals = np.diff(onset_timings)
    max_interval = int(np.max(time_intervals))  # Maximum interval as integer for matrix dimensions
    
    # Initialize segmented data matrix with shape (m, n, l)
    num_stimulations = len(onset_timings) - 1  # Number of segments based on stimulation
    segmented_data = np.zeros((num_stimulations, max_interval, num_muscles))
    
    # Segment the EMG data for each muscle
    for muscle_idx in range(num_muscles):
        muscle_data = emg_data[muscle_idx]
        for stim_idx in range(num_stimulations):
            start_time = onset_timings[stim_idx]
            end_time = start_time + max_interval
            
            # Segment the EMG data for the current interval
            segment = muscle_data[start_time:end_time]
            # If segment is shorter than max_interval, pad with zeros
            if len(segment) < max_interval:
                segment = np.pad(segment, (0, max_interval - len(segment)), 'constant')
            segmented_data[stim_idx, :, muscle_idx] = segment
            
    return segmented_data

def visualize_emg_segments(segmented_data):
    """
    Visualize EMG segments as colormap images for each muscle.
    
    Parameters:
        segmented_data (np.array): 3D array of segmented EMG data (m x n x l).
    """
    num_muscles = segmented_data.shape[2]
    
    for muscle_idx in range(num_muscles):
        plt.figure()
        plt.imshow(segmented_data[:, :, muscle_idx], aspect='auto', cmap='viridis')
        plt.colorbar()
        plt.title(f'EMG Segments for Muscle {muscle_idx + 1}')
        plt.xlabel('Time (ms)')
        plt.ylabel('Stimulation Pulsation')
        plt.show()

# Example usage
# emg_data is a list where each element is a 1D numpy array of EMG data for each muscle
# onset_timings is a list of stimulation onset times (in sample points or time indices)
# num_muscles is the number of muscles recorded
#emg_data = [np.random.randn(10000) for _ in range(5)]  # Example EMG data for 5 muscles

# Real code
file_path = "../Data/deidentified_Scrambled_EMG.mat"
data = load_pre_v7_3_mat(file_path)

Data = scipy.io.loadmat(r"../Data/Scrambled_EMG.mat")


#Data = load_pre_v7_3_mat("../Data/Scrambled_EMG.mat")
print(Data(1))


emg_data = data

#onset_timings = [0, 2000, 4000, 6000, 8000]  # Example onset timings

onset_timings = stim_timings(emg_data, fs) 

num_muscles = len(emg_data)

# Segment the EMG data
segmented_data = segment_emg_signals(emg_data, onset_timings, num_muscles)

# Visualize the segmented EMG data
visualize_emg_segments(segmented_data) 

TypeError: 'dict' object is not callable

In [56]:
from scipy.io import loadmat

Data = loadmat(r"../Data/Scrambled_EMG.mat")
x = Data['org_data']
df=pd.DataFrame(x,columns=["LRF","LVL","LMH","LTA","LMG","LS","LGM","RRF","RVL","RMH","RTA","RMG","RS","RGM","Artifact"])
df.to_csv(r"../Data/scrambled_EMG.csv",index=False)

#print(type(x))

#print(Data.keys())
